In [ ]:
!unzip /content/drive/MyDrive/Ford_Motoras/f0b108a2507d11ed.zip -d dataset_2

In [ ]:
import os, re
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import sys
from nltk.stem import WordNetLemmatizer
np.set_printoptions(threshold=sys.maxsize)
from sklearn.metrics import f1_score
from bs4 import BeautifulSoup
from sklearn.preprocessing import LabelBinarizer

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
files = os.listdir('./dataset_2/dataset/train/')
train_files = files[:int(len(files)*0.8)]
valid_files = files[int(len(files)*0.8):]

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('./dataset_2/dataset/train.csv')
df.head()
dict(df['Product'].value_counts())
file_cat = dict(df[['File_Name','Product']].values)

classes = {'AI': 0,
 'allopathy': 1,
 'Truck': 2,
 'Bike': 3,
 'Medical devices': 4,
 'Organicfood': 5,
 'Kids toys': 6,
 'Gaming hardware': 7,
 'Solar': 8,
 'Fuel': 9,
 'antivirus-security': 10,
 'Hydro': 11,
 'charcoal': 12}
df2 = pd.DataFrame({'filename':train_files,'class':None})
df2['class'] = df2['filename'].apply(lambda x: file_cat[x])
df2['class'].value_counts()
df2['class'] = df2['class'].apply(lambda x: classes[x])


biner = LabelBinarizer()
y_true = biner.fit_transform(df2['class'])

In [ ]:
len(valid_files), len(train_files)

(2691, 10761)

In [ ]:
tokener = Tokenizer(oov_token='<OOV>')
lemmatizer = WordNetLemmatizer()

In [ ]:
def clean_text(
    string: str, 
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',) -> str:
    """
    A method to clean text 
    """
    stop_words = {'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during',
     'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into',
      'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves',
       'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 
       'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 
       'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because',
        'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only',
         'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing',
          'it', 'how', 'further', 'was', 'here', 'than',''} 
    # Cleaning the urls
    string = re.sub(r'https?://\S+|www\.\S+', '', string)

    # Cleaning the html elements
    string = re.sub(r'<.*?>', '', string)
    string = re.sub(r'\S*\d+\S*', '', string)

    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 

    # Converting the text to lower
    string = string.lower()
    
    # Removing stop words
    string = ' '.join([lemmatizer.lemmatize(word) for word in string.split() if ((word not in stop_words) and len(word)>2)])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string      


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
X_train = []
for i in range(len(train_files)): #len(train_files)
    with open(f"./dataset_2/dataset/train/{train_files[i]}",'r') as f:
        text1 = f.read()
        soup = BeautifulSoup(text1,'lxml')
        text1 = soup.find(lambda tag:tag.name=="p" and "Company background" in tag.text).find_next('p')

        text1 = clean_text(text1.text).split(' ') 
        # print(text1)       
        tokener.fit_on_texts(text1)
        X_train.append(text1)
        

In [ ]:
max_len = np.max([len(i) for i in X_train])
print(max_len)

183


In [ ]:
# Creating the padded matrices
X_train_NN = tokener.texts_to_sequences(X_train)
X_train_NN = pad_sequences(X_train_NN, maxlen=max_len,padding='post', truncating='post')

In [ ]:
X_train_NN.shape

(10761, 183)

In [ ]:
unique_count = len(tokener.word_counts)
print(unique_count)

9174


In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional ,Flatten

# embed_dim = 128
# model = Sequential()
# model.add(Embedding(
#   input_dim=unique_count+2, 
#   output_dim=embed_dim, 
#   input_length=X_train_NN.shape[1]))
# model.add(Bidirectional(LSTM(256,return_sequences=True)))
# model.add(Bidirectional(LSTM(128,return_sequences=False)))
# model.add(Flatten())
# model.add(Dense(64,activation='relu'))
# model.add(Dense(13,activation='softmax'))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional ,Flatten

embed_dim = 128
model = Sequential()
model.add(Embedding(
  input_dim=unique_count+1, 
  output_dim=embed_dim, 
  input_length=X_train_NN.shape[1]))
model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(Bidirectional(LSTM(128,return_sequences=True)))
model.add(Bidirectional(LSTM(72,return_sequences=False)))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(13,activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
optimzer = Adam(learning_rate=0.001)
model.compile(loss = 'categorical_crossentropy', optimizer=optimzer)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
class_labels=[0,1,2,3,4,5,6,7,8,9,10,11,12]
weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=df2['class'])

In [ ]:
class_weights = dict(zip(class_labels,weights))

15+ 20 + 10+20+25+35

In [ ]:
class_weights

{0: 0.7064846416382252,
 1: 0.7961538461538461,
 2: 0.7638376383763837,
 3: 0.8313253012048193,
 4: 0.9672897196261683,
 5: 1.0247524752475248,
 6: 1.2034883720930232,
 7: 1.0952380952380953,
 8: 1.2621951219512195,
 9: 1.2248520710059172,
 10: 1.2105263157894737,
 11: 1.1965317919075145,
 12: 1.2621951219512195}

In [ ]:
model.fit(x = X_train_NN,y = y_true,epochs=45)

Epoch 1/45
337/337 [==============================] - 27s 59ms/step - loss: 1.2953
Epoch 2/45
337/337 [==============================] - 20s 60ms/step - loss: 0.6947
Epoch 3/45
337/337 [==============================] - 20s 58ms/step - loss: 0.5689
Epoch 4/45
337/337 [==============================] - 19s 57ms/step - loss: 0.5117
Epoch 5/45
337/337 [==============================] - 21s 63ms/step - loss: 0.4982
Epoch 6/45
337/337 [==============================] - 18s 55ms/step - loss: 0.4684
Epoch 7/45
337/337 [==============================] - 19s 56ms/step - loss: 0.4544
Epoch 8/45
337/337 [==============================] - 20s 58ms/step - loss: 0.4392
Epoch 9/45
337/337 [==============================] - 21s 62ms/step - loss: 0.4259
Epoch 10/45
337/337 [==============================] - 21s 63ms/step - loss: 0.4234
Epoch 11/45
337/337 [==============================] - 22s 66ms/step - loss: 0.4059
Epoch 12/45
337/337 [==============================] - 21s 62ms/step - loss: 0.4002
E

In [ ]:
model.save('model1.h5')

In [ ]:
df = pd.read_csv('./dataset_2/dataset/train.csv')
df.head()
file_cat = dict(df[['File_Name','Product']].values)
classes = {'AI': 0,
 'allopathy': 1,
 'Truck': 2,
 'Bike': 3,
 'Medical devices': 4,
 'Organicfood': 5,
 'Kids toys': 6,
 'Gaming hardware': 7,
 'Solar': 8,
 'Fuel': 9,
 'antivirus-security': 10,
 'Hydro': 11,
 'charcoal': 12}

df2 = pd.DataFrame({'filename':valid_files,'class':None})
df2['class'] = df2['filename'].apply(lambda x: file_cat[x])
df2['class'].value_counts()
df2['class'] = df2['class'].apply(lambda x: classes[x])
y_valid = biner.transform(df2['class'])

In [ ]:
X_valid = []
for i in range(len(valid_files)): #len(train_files)
    with open(f"./dataset_2/dataset/train/{valid_files[i]}",'r') as f:
        text = f.read()
        text = clean_text(text).split(' ')        
        # tokener.fit_on_texts(text)
        X_valid.append(text)

In [ ]:
# Creating the padded matrices
X_valid_NN = tokener.texts_to_sequences(X_valid)
X_valid_NN = pad_sequences(X_valid_NN, maxlen=max_len)
output_array = model.predict(X_valid_NN)

85/85 [==============================] - 2s 28ms/step


In [ ]:
out = biner.inverse_transform(output_array)
act = biner.inverse_transform(y_valid)
f1_score(out,act,average = 'weighted') * 100

19.37881399770392

## Testing

In [ ]:
test_files = pd.read_csv('./dataset_2/dataset/test.csv')['File_Name'].tolist()

In [ ]:
X_test = []
for i in range(len(test_files)): #len(train_files)
    with open(f"./dataset_2/dataset//test/{test_files[i]}",'r') as f:
        text = f.read()
        text = clean_text(text).split(' ')        
        # tokener.fit_on_texts(text)
        X_test.append(text)
# Creating the padded matrices
X_test_NN = tokener.texts_to_sequences(X_test)
X_test_NN = pad_sequences(X_test_NN, maxlen=max_len)


In [ ]:
output_array = model.predict(X_test_NN)
out = biner.inverse_transform(output_array)

75/75 [==============================] - 1s 19ms/step


In [ ]:
sub = pd.DataFrame({'File_Name':test_files,'Product':out})
sub.head()

,File_Name,Product
0,PROJ0x6c6.html,0
1,PROJ0x6606.html,0
2,PROJ0x86d0.html,0
3,PROJ0x93b4.html,7
4,PROJ0x2cd6.html,0


In [ ]:
classes_ = {0:'AI',
 1:'allopathy',
 2:'Truck',
 3:'Bike',
 4:'Medical devices',
 5:'Organicfood',
 6:'Kids toys',
 7:'Gaming hardware',
 8:'Solar',
 9:'Fuel',
 10:'antivirus-security',
 11:'Hydro',
 12:'charcoal'
 }
sub['Product'] = sub['Product'].apply(lambda x: classes_[x])
sub.to_csv('sub1_7.csv',index=False)